In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_json,col
from pyspark.sql.types import *
from os.path import abspath

In [4]:
spark = SparkSession.builder\
        .master('local[*]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4040')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '5g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

In [4]:
!pip install pandas
!pip install folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 151.4 kB/s eta 0:00:00a 0:00:01


In [5]:
import pandas as pd

In [6]:
import folium

In [7]:
import requests

## 3. The Data

data from https://github.com/davidmegginson/ourairports-data/tree/02e9d743f1dad41a9950191f080c6a169892d9e9

In [8]:
!wget -nc https://raw.githubusercontent.com/davidmegginson/ourairports-data/02e9d743f1dad41a9950191f080c6a169892d9e9/airports.csv

--2023-11-18 11:35:32--  https://raw.githubusercontent.com/davidmegginson/ourairports-data/02e9d743f1dad41a9950191f080c6a169892d9e9/airports.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10964897 (10M) [text/plain]
Saving to: ‘airports.csv’

airports.csv        100%[===================>]  10.46M  2.08MB/s    in 5.0s    

2023-11-18 11:35:40 (2.08 MB/s) - ‘airports.csv’ saved [10964897/10964897]



In [9]:
!wget -nc https://raw.githubusercontent.com/davidmegginson/ourairports-data/02e9d743f1dad41a9950191f080c6a169892d9e9/countries.csv

--2023-11-18 11:35:45--  https://raw.githubusercontent.com/davidmegginson/ourairports-data/02e9d743f1dad41a9950191f080c6a169892d9e9/countries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24443 (24K) [text/plain]
Saving to: ‘countries.csv’

countries.csv       100%[===================>]  23.87K  62.3KB/s    in 0.4s    

2023-11-18 11:35:48 (62.3 KB/s) - ‘countries.csv’ saved [24443/24443]



In [1]:
!wget -nc https://raw.githubusercontent.com/kdulep/gb-diplom-gis/main/countries-boundaries.csv

File ‘countries-boundaries.csv’ already there; not retrieving.



In [8]:
data = spark.read.csv(path="countries-boundaries.csv", sep=",", header=True)

In [9]:
country="RU"

In [10]:
#data.count()
df1=data.filter(data['country']==country)
datac=df1.toPandas()
datac

,country,latitude,longitude,name
0,RU,61.52401,105.318756,Russia


In [ ]:
#df1 = data.select(["column1", "column2", "column3"])

In [11]:
dataair = spark.read.csv(path="airports.csv", sep=",", header=True)
dataair.show()

+------+-----+-------------+--------------------+------------------+-------------------+------------+---------+-----------+----------+------------+-----------------+--------+---------+----------+--------------------+--------------+--------+
|    id|ident|         type|                name|      latitude_deg|      longitude_deg|elevation_ft|continent|iso_country|iso_region|municipality|scheduled_service|gps_code|iata_code|local_code|           home_link|wikipedia_link|keywords|
+------+-----+-------------+--------------------+------------------+-------------------+------------+---------+-----------+----------+------------+-----------------+--------+---------+----------+--------------------+--------------+--------+
|  6523|  00A|     heliport|   Total RF Heliport|         40.070985|         -74.933689|          11|       NA|         US|     US-PA|    Bensalem|               no|    K00A|     NULL|       00A|https://www.pennd...|          NULL|    NULL|
|323361| 00AA|small_airport|Aero B R

In [ ]:
#dataair.write.saveAsTable('airports')

In [12]:
datacountry = spark.read.csv(path="countries.csv", sep=",", header=True)
datacountry.show()

+------+----+--------------------+---------+--------------------+--------------------+
|    id|code|                name|continent|      wikipedia_link|            keywords|
+------+----+--------------------+---------+--------------------+--------------------+
|302672|  AD|             Andorra|       EU|https://en.wikipe...|   Andorran airports|
|302618|  AE|United Arab Emirates|       AS|https://en.wikipe...|UAE,مطارات في الإ...|
|302619|  AF|         Afghanistan|       AS|https://en.wikipe...|                NULL|
|302722|  AG| Antigua and Barbuda|       NA|https://en.wikipe...|   Antiguan airports|
|302723|  AI|            Anguilla|       NA|https://en.wikipe...|                NULL|
|302673|  AL|             Albania|       EU|https://en.wikipe...|   Albanian airports|
|302620|  AM|             Armenia|       AS|https://en.wikipe...|                NULL|
|302556|  AO|              Angola|       AF|https://en.wikipe...|    Angolan airports|
|302615|  AQ|          Antarctica|       AN

In [ ]:
#datacountry.write.saveAsTable('countries')

In [ ]:
#df1 = data.select(["column1", "column2"])

In [ ]:
#cols_description = df1.filter(df1['column1'] == 'Units')

In [ ]:
#df1 = df1.join(other=cols_description, on=['column1'], how='left_anti')

In [ ]:
#df1.write.saveAsTable('hive_table')

In [14]:
datacountry.createOrReplaceTempView("countries")
dataair.createOrReplaceTempView("airports")

In [15]:
sparkdf=spark.sql(f'select a.name as airport_name, a.latitude_deg, a.longitude_deg, a.wikipedia_link, a.home_link from countries c inner join airports a ON(c.code=a.iso_country) WHERE a.iso_country="{country}";')
df = sparkdf.toPandas()

In [ ]:
#for index, row in df.iterrows():
#    print(row["airport_name"], row['latitude_deg'], row['longitude_deg'], row["home_link"] )
#for ind in ourdf:
#    print(ourdf['name'][ind])

In [16]:
m = folium.Map([datac['latitude'],datac['longitude']], tiles="cartodbpositron")
geojson_data = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/world_countries.json"
).json()

folium.GeoJson(geojson_data, name="hello world").add_to(m)

folium.LayerControl().add_to(m)

for index, row in df.iterrows():
#    print(row["airport_name"], row['latitude_deg'], row['longitude_deg'] )

     folium.Marker(location=[row['latitude_deg'], row['longitude_deg']], tooltip=row["airport_name"], popup=row["home_link"],
    icon=folium.Icon(color="green"),
).add_to(m)

/opt/conda/lib/python3.11/site-packages/folium/utilities.py:94: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/opt/conda/lib/python3.11/site-packages/folium/utilities.py:100: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/opt/conda/lib/python3.11/site-packages/folium/utilities.py:102: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(x) for x in coords]


In [17]:
sw = df[['latitude_deg', 'longitude_deg']].min().values.tolist()
ne = df[['latitude_deg', 'longitude_deg']].max().values.tolist()

m.fit_bounds([sw, ne]) 
m

In [ ]:
m.save("airports-data.html")